# <b><span style="color:#ADF88B">10 x 3 sec audio files to 30 sec audio file </span> </b>

Multiple tests were conducted to evaluate Whisper's response in this scenario:

1. <span style="color:#58F2B3"> Grouping by score: Each score was grouped before transcription. Sorted by file name.</span>
2. <span style="color:#58F2B3"> Grouping by score: Each score was grouped before transcription. Mixed file names.</span>

3. <span style="color:#00D5F7"> Grouping by person: Each person's independent words were grouped, and sorted by score.</span>
4. <span style="color:#00D5F7"> Random shuffling of data before transcription.</span>

The results were analyzed using <span style="color:#F9F871">Word Error Rate (WER)</span> and  <span style="color:#F9F871">Character Error Rate (CER)</span> scores to assess the performance of the models.

In [2]:
import self_made_functions as smf
from prettytable import PrettyTable
from transformers import pipeline
import matplotlib.pyplot as plt
from pydub import AudioSegment 
import pandas as pd 
import numpy as np
import whisper
import jiwer
import os

2024-07-06 13:51:00.652295: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-06 13:51:01.823843: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
df_assessment, _ = smf.get_correct_df()
df= df_assessment
s,e = 0,10

In [16]:
# def per_wer_input_name(df:pd.DataFrame, s:int=0, e:int=10):

concat_rows = df.iloc[s:e]  # Rows to concatenate
input_words = list(concat_rows.Word.values) # get input string

phone_score_lst = [] # list of phone scores to calculate PER
wrong_words = 0 # count of wrong words to calculate WER

for row in concat_rows.itertuples():
    phone_score_lst.append(row.pronScores)
    
    if row.pronScores.count('1') > 0:
        wrong_words += 1   

# --- Word Error Rate (WER) --- #
number_of_words = len(phone_score_lst)
wer = wrong_words/number_of_words # Correct Word Error Rate (WER)

# --- Calculate Phone Error Rate (PER) --- #
# Make phone scores to one string, and then split into a list
phone_scores = (' '.join(phone_score_lst)).split(' ') 
incorrect_phones = phone_scores.count('0')
phone_length = len(phone_scores)
per = incorrect_phones/phone_length


10


In [13]:
incorrect_phones

17

In [12]:
phone_scores.split(' ').count('0')

17

In [ ]:
    pron_lst = (' '.join(pron_lst)) # String of all the 0 adn 1s
    target_per = pron_lst.count('1')/len(pron_lst) # Correct Character Error Rate (CER)
    return target_per, target_wer, input_words



In [ ]:

def concat_audio(df:pd.DataFrame, save_directory:str, s:int=0, e:int=10, 
                wv_path:str = '/talebase/data/speech_raw/teflon_no/speech16khz/', id:str = None):
    concat_rows = df.iloc[s:e]  # Rows to concatenate
    
    combined_audio = AudioSegment.empty() # Empty audio file to concatenate the audio clips
    
    for row in concat_rows.itertuples():
        audio = AudioSegment.from_file(os.path.join(wv_path, row[1]))
        combined_audio += audio
    
    if id:
        audio_name =f'{id}_words_{s}_{e}.wav'
    else:
        audio_name = f'words_{s}_{e}.wav'

    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
        print(f"Directory {save_directory} created")

    # For store filer! Ikke lagre dem direkte i git repo, men på serveren
    audio_directory = os.path.join(save_directory, audio_name)
    # Save the combined audio
    if not os.path.exists(audio_directory):
        combined_audio.export(audio_directory, format="wav")   
    
    return  audio_name